In [189]:
#import libraries
import pandas as pd
#read data
df = pd.read_csv('/Users/scottgleave/Downloads/Parking_Tickets.csv')
df['Date'] = df.DateIssued.str[:10]
df['Time'] = df.DateIssued.str[11:-5]
# Hormat the Hour column to have padded zeroes
df['Hour'] = pd.to_datetime(df['Time'], format= '%H:%M:%S' ).dt.hour
df['Hour'] = df.Hour.map("{:02}".format)
# Add column that takes into account if this the time is midnight, then output the hour of the TimeIssued column
df['Hour'] = np.where(df.eval("Time == '00:00:00'"), df.TimeIssued.str[:2], df.Hour)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [190]:
#read data to tickets df and create column for year
tickets = df
tickets['year'] = tickets.Date.str[:4]

In [243]:
#find rows with appeals and create new df.  Count values for appeal status categories
appeal = tickets.dropna(subset=['AppealDate'])
appealcounts = pd.value_counts(appeal['AppealStatus'].values, sort=False)
appealcounts

pending      721
denied     13055
granted    13422
dtype: int64

In [193]:
#display percentages for appeal status categories
appeal.AppealStatus.value_counts(normalize=True)

granted    0.493492
denied     0.479999
pending    0.026509
Name: AppealStatus, dtype: float64

In [244]:
#what percent of appeals were successful by type of offense?

#get value counts for violationdescription
appealcountsbyoffense = pd.value_counts(appeal['ViolationDescription'].values, sort=False)
#newdataframe with only appeals that were granted
appealsgranted = appeal[(appeal['AppealStatus'] == 'granted')]
#get value counts for violationdescription for appeals that were granted
appealcountsgranted = pd.value_counts(appealsgranted['ViolationDescription'].values, sort=False)
#combine values count dfs and calculate percentage of appeals granted by violation description
appealgranteddf = pd.concat([appealcountsbyoffense, appealcountsgranted], axis=1)
appealgranteddf['perc'] = appealgranteddf[1]/appealgranteddf[0]
appealgranteddf.sort_values(by=['perc'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """


,0,1,perc
Blocking Drive,161,37,0.229814
15ft of Fire Hydrant,145,38,0.262069
No Parking Desig HR,102,28,0.274510
Void,13,4,0.307692
Occupying Adjoining Space,341,106,0.310850
12 Inches From Curb,202,63,0.311881
Bus or Taxi Zone,49,16,0.326531
LT Side to Curb,566,192,0.339223
Obstructing Traffic,143,50,0.349650
No Parking any time,2725,996,0.365505


In [245]:
#value counts for days when there were many appeals
appealbyday = pd.value_counts(appeal['Date'].values, sort=True)
appealbyday.head()

2004-02-17    120
2004-02-18    107
2003-02-25     92
2003-02-24     88
2004-02-19     81
dtype: int64

In [200]:
#Create df to figure out how percentage of appeals granted changed over time
#value counts by year for total appeals
appealbyyear = pd.value_counts(appeal['year'].values, sort = False)
#value counts by year for appeals granted
appealgrantbyyear = pd.value_counts(appealsgranted['year'].values, sort = False)
#combine value count dfs and calculate yearly percentage granted
appealsuccessyear = pd.concat([appealbyyear, appealgrantbyyear], axis=1)
appealsuccessyear['perc'] = appealsuccessyear[1]/appealsuccessyear[0]
appealsuccessyear

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,perc
1999,241,133,0.551867
2000,1778,919,0.516873
2001,3109,1825,0.587005
2002,2346,660,0.281330
2003,2728,1081,0.396261
2004,3041,1444,0.474844
2005,2283,1068,0.467806
2006,1367,548,0.400878
2007,919,324,0.352557
2008,742,354,0.477089


In [201]:
#pendings by year
#new df for pending tickets, then perform value count
pending = tickets[(tickets['AppealStatus'] == 'pending')]
pendingbyyear = pd.value_counts(pending['year'].values, sort = True)
pendingbyyear

2004    124
2001     97
2003     83
2000     83
2005     78
2002     77
2006     69
2008     31
2014     24
2015     19
2007     19
2016     16
2017     12
1999      8
2009      5
2018      5
2010      3
2013      2
2011      2
2012      1
dtype: int64

In [246]:
#time between request and decision on appeal

#sort dataframe by date and remove last 10 rows to remove incorrect dates that cannot be converted to datetime
#and convert to datetime
tickets = tickets.sort_values(by=['Date'], ascending = True)
tickets = tickets[:-10]
tickets['Date'] = pd.to_datetime(tickets['Date'])
#sort dataframe by granted date, remove incorrect dates
appealgrantdf = tickets.dropna(subset=['AppealGrantedDate'])
appealgrantdf['tempcol'] = appealgrantdf.AppealGrantedDate.str[-4:]
appealgrantdf = appealgrantdf.sort_values(by=['tempcol'], ascending = False)
appealgrantdf = appealgrantdf[10:]
#sort dataframe by granted date, remove incorrect dates
appealgrantdf['tempcol'] = appealgrantdf.AppealDate.str[-4:]
appealgrantdf = appealgrantdf.sort_values(by=['tempcol'], ascending = False)
appealgrantdf = appealgrantdf[10:]
#convert formats to datetime
appealgrantdf['AppealDate'] = pd.to_datetime(appealgrantdf['AppealDate'])
appealgrantdf['AppealGrantedDate'] = pd.to_datetime(appealgrantdf['AppealGrantedDate'])
appealgrantdf = appealgrantdf.drop('tempcol', axis = 1)

#figure out date range
appealgrantdf['daylength'] = appealgrantdf['AppealGrantedDate'] - appealgrantdf['AppealDate']
appealgrantdf['daylength'].describe()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


count                       26474
mean      13 days 13:56:59.959205
std      126 days 00:30:33.861638
min           -722 days +00:00:00
25%               1 days 00:00:00
50%               3 days 00:00:00
75%               6 days 00:00:00
max            3164 days 00:00:00
Name: daylength, dtype: object

In [238]:
#only get tickets that have appeal granted date
#Convert to datetime
appealgrantdf = tickets.dropna(subset=['AppealGrantedDate'])
appealgrantdf['tempcol'] = appealgrantdf.AppealGrantedDate.str[-4:]
appealgrantdf = appealgrantdf.sort_values(by=['tempcol'], ascending = False)
appealgrantdf = appealgrantdf[10:]
appealgrantdf['tempcol'] = appealgrantdf.AppealDate.str[-4:]
appealgrantdf = appealgrantdf.sort_values(by=['tempcol'], ascending = False)
appealgrantdf = appealgrantdf[10:]
appealgrantdf['AppealDate'] = pd.to_datetime(appealgrantdf['AppealDate'])
appealgrantdf['AppealGrantedDate'] = pd.to_datetime(appealgrantdf['AppealGrantedDate'])
appealgrantdf = appealgrantdf.drop('tempcol', axis = 1)

#figure out date range
appealgrantdf['daylength'] = appealgrantdf['AppealGrantedDate'] - appealgrantdf['AppealDate']
appealgrantdf['daylength'].describe()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


count                       26474
mean      13 days 13:57:35.858578
std      126 days 00:30:31.196832
min           -722 days +00:00:00
25%               1 days 00:00:00
50%               3 days 00:00:00
75%               6 days 00:00:00
max            3164 days 00:00:00
Name: daylength, dtype: object

In [247]:
#cars successful with appeals multiple times

#create dataframe from granted, value count by license plate
successful = tickets[(tickets['AppealStatus'] == 'granted')]
successfulcount = pd.value_counts(successful['LicensePlateAnon'].values, sort=True)
successfulcount.head()

180337    37
125415    25
75826     23
16808     22
72071     21
dtype: int64

In [227]:
#percent appealed by year

#count value of tickets by year, add to appeals by year, and calculate percentage
ticketsbyyear = pd.value_counts(tickets['year'].values, sort = False)
percentappealtime = pd.concat([appealbyyear, ticketsbyyear], axis=1)
percentappealtime['perc'] = percentappealtime[0]/percentappealtime[1]
percentappealtime

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,perc
1999,241,11152,0.021610
2000,1778,45567,0.039019
2001,3109,42169,0.073727
2002,2346,24912,0.094171
2003,2728,26216,0.104059
2004,3041,29813,0.102002
2005,2283,26757,0.085323
2006,1367,19103,0.071559
2007,919,22526,0.040797
2008,742,17181,0.043187


In [242]:
#percentage breakdown of tickets with waiver by violation description
waiver = tickets.dropna(subset=['WaiverRequestDate'])
waiver.ViolationDescription.value_counts(normalize=True)

Overtime Parking                0.915846
Violation of Meter Ordinance    0.083591
Void                            0.000563
Name: ViolationDescription, dtype: float64